In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [2]:
#read in data
permits_df = pd.read_csv("../data/sidewalk-cafe-permits.csv")
#drop duplicates but there are none
permits_df.drop_duplicates()
#Rename columns to what we want to be in SQL Database
permits_df = permits_df.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})


In [3]:
#Changed Column names
permits_df.columns

Index(['permit_number', 'ACCOUNT NUMBER', 'SITE NUMBER', 'LEGAL NAME',
       'business_name', 'issued_date', 'expiration_date', 'PAYMENT DATE',
       'address', 'ADDRESS NUMBER START', 'ADDRESS NUMBER', 'STREET DIRECTION',
       'STREET', 'STREET TYPE', 'CITY', 'STATE', 'ZIP CODE', 'WARD',
       'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'latitude', 'longitude',
       'location', 'Community Areas', 'Zip Codes', 'Boundaries - ZIP Codes',
       'Census Tracts', 'Wards'],
      dtype='object')

In [11]:
# Make dataframe of only the columns we want to bring into SQL
import_permits_df = permits_df[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]


In [17]:
#Find any null values
import_permits_df.isnull().values.any()
#drop the null values
import_permits_df = import_permits_df.dropna()


In [20]:
import_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1142666,TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,7958 W BELMONT AVE,41.937859,-87.826255
1,1142290,Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,5357 N ASHLAND AVE,41.979717,-87.669565
2,1142132,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,333 N MICHIGAN AVE,41.887543,-87.624407
3,1142101,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,1732 N MILWAUKEE AVE,41.912730,-87.681327
4,1142100,THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2427-2431 N MILWAUKEE AVE,41.925388,-87.701170


In [29]:
#Read in second CSV file which contains data for 2020
current_permits = pd.read_csv("../data/current_permits.csv")
current_permits.head()
#Rename columns to bring into SQL
current_permits = current_permits.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})


In [30]:
#Create import Dataframe with only columns we want to bring into SQL
import_current_permits_df = current_permits[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]
import_current_permits_df


,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777
...,...,...,...,...,...,...,...
1090,1148345,FRANKLIN ROOM,04/09/2021,02/28/2022,675 N FRANKLIN ST,41.894589,-87.635506
1091,1148368,Kraken Sushi & Beyond,04/13/2021,02/28/2022,4022 N WESTERN AVE,41.954759,-87.688635
1092,1148330,AVEC II,04/13/2021,02/28/2022,141 W ERIE ST,41.893900,-87.632732
1093,1148380,FONS EMPANADAS,04/13/2021,02/28/2022,2566 N CLARK ST,41.929403,-87.642899


In [33]:
#find any null values 
import_current_permits_df.isnull().sum()
#drop the null values
import_current_permits_df = import_current_permits_df.dropna()

In [35]:
import_current_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777


In [24]:
# After creating the 2 tables in posgresql (sql) via pgAdmin 4:
# Connect to local database
connection_string = "postgres:broken@localhost:5432/chicago_cafe_permits"
engine = create_engine(f'postgresql://{connection_string}')
# ensure table names appear
engine.table_names()

['past_permits', 'current_permits']

In [25]:
# Load past_permits dataframe into sql database
import_permits_df.to_sql(name='past_permits', con=engine, if_exists='append', index=False)

In [36]:
# Load current_permits dataframe into sql database
import_current_permits_df.to_sql(name='current_permits', con=engine, if_exists='append', index=False)

In [38]:
import_current_permits_df
import_permits_df

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1142666,TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,7958 W BELMONT AVE,41.937859,-87.826255
1,1142290,Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,5357 N ASHLAND AVE,41.979717,-87.669565
2,1142132,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,333 N MICHIGAN AVE,41.887543,-87.624407
3,1142101,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,1732 N MILWAUKEE AVE,41.912730,-87.681327
4,1142100,THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2427-2431 N MILWAUKEE AVE,41.925388,-87.701170
...,...,...,...,...,...,...,...
17993,1003612,THE REVEL ROOM,2001-08-01T00:00:00.000,2001-11-01T00:00:00.000,1566 N MILWAUKEE AVE,41.909880,-87.676914
17994,1002993,O'NEIL'S BAR & GRILL,2001-04-04T00:00:00.000,2001-11-01T00:00:00.000,152 E ONTARIO ST,41.893460,-87.623374
17995,1002652,CAFE MAURO,2001-05-07T00:00:00.000,2001-11-01T00:00:00.000,3633 W 26TH ST,41.844259,-87.715946
17996,1001307,Rick's Cafe-Casablanca,2001-06-18T00:00:00.000,2001-11-01T00:00:00.000,3915 N SHERIDAN RD,41.953158,-87.654359
